# Load HD binary files coming out of iMOD/MF6 simulaton and POP them

GXG and other averages are calculated and saved as TIFs.
Function developed for NBr43, but made into a geo.py function right after I was finished.

# 0. Initial

## 0.0. Imports

In [1]:
import WS_Mdl.utils as U
import WS_Mdl.utils_imod as UIM
import WS_Mdl.calcs as C
import WS_Mdl.geo as G

In [2]:
import importlib as IL
IL.reload(U)
IL.reload(UIM)
IL.reload(G)

<module 'WS_Mdl.geo' from 'G:\\code\\WS_Mdl\\geo.py'>

In [3]:
import numpy as np
import xarray as xr

In [4]:
import os
from os import listdir as LD, makedirs as MDs
from os.path import join as PJ, basename as PBN, dirname as PDN, exists as PE
import shutil as sh
import pandas as pd
from datetime import datetime as DT
import matplotlib.pyplot as plt
from pathlib import Path
import re

In [5]:
import imod

## 0.1. Options

In [6]:
U.set_verbose(False)

In [7]:
MdlN = 'NBr40'

# Load paths and variables from PRJ & INI
d_Pa = U.get_MdlN_Pa(MdlN)
Pa_PRJ = d_Pa['PRJ']
Dir_PRJ = PDN(Pa_PRJ)
d_INI = U.INI_to_d(d_Pa['INI'])
Xmin, Ymin, Xmax, Ymax, cellsize, N_R, N_C = U.Mdl_Dmns_from_INI(d_Pa['INI'])
SP_date_1st, SP_date_last= [DT.strftime(DT.strptime(d_INI[f'{i}'], '%Y%m%d'), '%Y-%m-%d') for i in ['SDATE', 'EDATE']]
IDT = int(d_INI['IDT'])
dx = dy = float(d_INI['CELLSIZE'])

In [8]:
Pa_PoP = d_Pa['PoP']

In [9]:
l_years = [i for i in range(1993, 1993+8+1)]
l_Ls = [i for i in range(1,11+1,2)]
l_Ls, l_years

([1, 3, 5, 7, 9, 11], [1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001])

# 1. Load Bin HD file

## 1.0. Load

In [10]:
DA_HD = imod.mf6.open_hds(hds_path=d_Pa['Out_HD_Bin'], 
                       grb_path=PJ( d_Pa['Sim_In'], 'dis.dis.grb'))

## 1.1 Trim

In [11]:
# Reconstruct time coordinate
dates = pd.date_range(start=SP_date_1st, periods=DA_HD.time.size, freq=f'{IDT}D')
DA_HD = DA_HD.assign_coords(time=dates) # Assign to DA_HD

In [12]:
# Check the result
DA_HD.time

<xarray.DataArray 'time' (time: 3263)> Size: 26kB
array(['1993-01-01T00:00:00.000000000', '1993-01-02T00:00:00.000000000',
       '1993-01-03T00:00:00.000000000', ..., '2001-12-05T00:00:00.000000000',
       '2001-12-06T00:00:00.000000000', '2001-12-07T00:00:00.000000000'],
      shape=(3263,), dtype='datetime64[ns]')
Coordinates:
    dx       float64 8B 25.0
    dy       float64 8B -25.0
  * time     (time) datetime64[ns] 26kB 1993-01-01 1993-01-02 ... 2001-12-07

In [13]:
DA_HD = DA_HD.where(DA_HD.time.dt.year.isin(l_years), drop=True).sel(layer=l_Ls) # Select specific years and layers

# 2. GXG

## 2.0. Calculate GXG

In [14]:
d_GXG = {}

In [15]:
for L in l_Ls:
    DA_HD_L = DA_HD.sel(layer=L)
    GXG = imod.evaluate.calculate_gxg(DA_HD_L).load()
    GXG = GXG.rename_vars({var: var.upper() for var in GXG.data_vars})

    # Get N_years
    N_years_GXG = np.unique(GXG.N_YEARS_GXG.values).max()
    N_years_GVG = np.unique(GXG.N_YEARS_GVG.values).max()

    # Calculate GHG - GLG
    GXG['GHG_m_GLG'] = GXG['GHG'] - GXG['GLG']
    GXG = GXG[['GHG', 'GLG', 'GHG_m_GLG', 'GVG']]

    # Collect results
    for var in GXG.data_vars:
        if var not in d_GXG:
            d_GXG[var] = []
        d_GXG[var].append(GXG[var])

In [16]:
# Concatenate
for var in d_GXG:
    if isinstance(d_GXG[var], list):
        d_GXG[var] = xr.concat(d_GXG[var], dim=pd.Index(l_Ls, name='layer'))

In [17]:
d_Pa.keys()

dict_keys(['imod_V', 'Mdl', 'MdlN', 'Pa_Mdl', 'Smk_temp', 'In', 'PoP', 'code', 'pixi', 'coupler_Exe', 'MF6_DLL', 'MSW_DLL', 'INI', 'BAT', 'PRJ', 'Smk', 'Sim', 'Pa_MdlN', 'MF6', 'MSW', 'TOML', 'TOML_iMOD5', 'LST_Sim', 'LST_Mdl', 'NAM_Sim', 'NAM_Mdl', 'Sim_In', 'Sim_Out', 'SFR', 'Out_HD', 'Out_HD_Bin', 'DIS_GRB', 'PoP_Out_MdlN', 'MM'])

## 2.1. Save

In [18]:
# Save to TIF
MDs(PJ(Pa_PoP, 'Out', MdlN, 'GXG'), exist_ok=True)

In [19]:
d_GXG.keys()

dict_keys(['GHG', 'GLG', 'GHG_m_GLG', 'GVG'])

In [22]:
for K, GXG in d_GXG.items():
    L_min, L_max = GXG.layer.values.min(), GXG.layer.values.max()

    Pa_Out = PJ(Pa_PoP, 'Out', MdlN, 'GXG', f'{K}_L{L_min}-{L_max}_{MdlN}.tif')

    d_MtDt = {
        f'{K}_L{L_min}-{L_max}_{MdlN}': {
            'AVG': float(GXG.mean().values),
            'coordinates': GXG.coords,
            'N_years': N_years_GVG if K == 'GVG' else N_years_GXG,
            'variable': os.path.splitext(PBN(Pa_Out))[0],
            'details': f"{MdlN} {K} calculated from (path: {d_Pa['Out_HD_Bin']}), via function described in: https://deltares.github.io/imod-python/api/generated/evaluate/imod.evaluate.calculate_gxg.html",
        }
    }

    G.DA_to_MBTIF(GXG, Pa_Out, d_MtDt, _print=False)